In [ ]:
import warnings
warnings.filterwarnings("ignore")
import numpy
import pandas
import matplotlib.pyplot as plt
import plotly.express as px
import pathlib
from glob import glob
import os
import PIL
import imageio
from scipy.signal import convolve2d
import matplotlib.image as mpimg
from tensorflow.keras.preprocessing import image

import tensorflow as tf
from tensorflow import keras

from sklearn import metrics
import seaborn

# Reading Directory

In [ ]:
from google.colab import drive
drive.mount('/content/drive')
train_path='/content/drive/MyDrive/APOD_Train/'
valid_path='/content/drive/MyDrive/APOD_Valid/'

# Image Processing

In [ ]:
trpth=os.listdir(train_path)
valpth=os.listdir(valid_path)
trn_img=[]
val_img=[]
cats_img=['Fake','Real']
for i in trpth:
    trn_img.append(len(os.listdir(os.path.join(train_path,i))))
for i in valpth:
    val_img.append(len(os.listdir(os.path.join(valid_path,i))))
tot=[sum(trn_img),sum(val_img)]
trn_img.append(tot[0])
val_img.append(tot[1])
cats_img.append("Total")
piedata=pandas.DataFrame({"Category":cats_img,"Train":trn_img,"Valid":val_img})
piedata

In [ ]:
fig=px.pie(piedata,names="Category",values="Train",color="Category",title="Number of Images Train Set",height=400,width=400,
           color_discrete_sequence=px.colors.qualitative.Vivid)
fig.update_traces(textposition='inside',textinfo='percent+label+value')
fig.update_layout(
    font=dict(
        family="Times New Roman, Bold",
        size=20,
        color="black"
    )
)
fig.show()

In [ ]:
train_dir = pathlib.Path(train_path)

trn_real = list(train_dir.glob("Real/*"))
trn_fake = list(train_dir.glob("Fake/*"))

In [ ]:
img1 = PIL.Image.open(trn_real[30])
plt.figure(figsize=(4,4))
plt.title("Real Astronimical Object",fontsize=15,color="#357EC7")
plt.imshow(img1)

img2 = PIL.Image.open(trn_fake[30])
plt.figure(figsize=(4,4))
plt.title("Fake Astronimical Object",fontsize=15,color="#357EC7")
plt.imshow(img2)

# Feature Extraction

In [ ]:
train_datagen = image.ImageDataGenerator(rescale=1. / 255,
                                   rotation_range=50,
                                   width_shift_range=0.15,
                                   height_shift_range=0.15,
                                   shear_range=0.3,
                                   zoom_range=0.3,
                                   vertical_flip=True)

val_datagen = image.ImageDataGenerator(rescale=1. / 255)

bsiz = 32
img_h,img_w = 128,128
labels=os.listdir(train_path)
print(labels)

In [ ]:
data_aug_train = train_datagen.flow_from_directory(
        train_path,
        classes=labels,
        target_size=(img_h, img_w),
        batch_size=bsiz,
        class_mode='categorical')

In [ ]:
data_aug_val = val_datagen.flow_from_directory(
        valid_path,
        classes=labels,
        target_size=(img_h,img_w),
        batch_size=bsiz,
        class_mode='categorical')

# Model building

In [1]:
def Model_Pretrained(dptlm,mdlnm):
    mdl_trl = dptlm
    inpinslyr = keras.layers.Input(shape=(img_h,img_w, 3))
    hddninslyr = keras.layers.Conv2D(3, (3, 3), padding='same')(inpinslyr)
    hddninslyr = mdl_trl(hddninslyr)
    hddninslyr = keras.layers.GlobalAveragePooling2D()(hddninslyr)
    hddninslyr = keras.layers.BatchNormalization()(hddninslyr)
    hddninslyr = keras.layers.Dropout(0.5)(hddninslyr)
    hddninslyr = keras.layers.Dense(256, activation='relu')(hddninslyr)
    hddninslyr = keras.layers.BatchNormalization()(hddninslyr)
    hddninslyr = keras.layers.Dropout(0.5)(hddninslyr)
    outinslyr = keras.layers.Dense(2,activation = 'softmax')(hddninslyr)
    deepemo = keras.models.Model(inpinslyr,outinslyr, name="{}".format(mdlnm))
    deepemo.compile(loss='categorical_crossentropy', metrics=['accuracy'])
    deepemo.summary()
    return deepemo